In [1]:
##script to run caiman cnmfe 
#pass folder containing motion corrected mmpap file concactenated in C order as sys arg 1
#motion corrected mmpap file concactenated in C order as sys arg 2

##!/Users/johnmarshall/miniconda3/envs/caiman/bin/python3
try:
    get_ipython().magic(u'load_ext autoreload')
    get_ipython().magic(u'autoreload 2')
    get_ipython().magic(u'matplotlib qt')
except:
    pass

import logging
import matplotlib.pyplot as plt
import numpy as np

logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",
                    level=logging.DEBUG)

import caiman as cm
from caiman.source_extraction import cnmf
from caiman.utils.utils import download_demo
from caiman.utils.visualization import inspect_correlation_pnr
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
import cv2
import sys
import h5py

try:
    cv2.setNumThreads(0)
except:
    pass
import bokeh.plotting as bpl
bpl.output_notebook()

#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

Loading BokehJS ...

In [5]:
## path to motion corrected tif tif file
folder = '/Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/'
mc_file = 'memmap__d1_480_d2_752_d3_1_order_C_frames_202_.mmap'
#folder = sys.arvg[1]
#mc_file = sys.arvg[2]
memory_map_file = folder+mc_file

In [6]:
print(memory_map_file)

/Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap__d1_480_d2_752_d3_1_order_C_frames_202_.mmap


In [7]:
# load memory mappable file
Yr, dims, T = cm.load_memmap(memory_map_file)
images = Yr.T.reshape((T,) + dims, order='F')

#
opts = params.CNMFParams(params_dict={})
#
bord_px = 0
# parameters for source extraction and deconvolution
p = 1               # order of the autoregressive system
K = None            # upper bound on number of components per patch, in general None
gSig = (3, 3)       # gaussian width of a 2D gaussian kernel, which approximates a neuron
gSiz = (13, 13)     # average diameter of a neuron, in general 4*gSig+1
Ain = None          # possibility to seed with predetermined binary masks
merge_thresh = .7   # merging threshold, max correlation allowed
rf = 40             # half-size of the patches in pixels. e.g., if rf=40, patches are 80x80
stride_cnmf = 20    # amount of overlap between the patches in pixels
#                     (keep it at least large as gSiz, i.e 4 times the neuron size gSig)
tsub = 2            # downsampling factor in time for initialization,
#                     increase if you have memory problems
ssub = 1            # downsampling factor in space for initialization,
#                     increase if you have memory problems
#                     you can pass them here as boolean vectors
low_rank_background = None  # None leaves background of each patch intact,
#                     True performs global low-rank approximation if gnb>0
gnb = 0             # number of background components (rank) if positive,
#                     else exact ring model with following settings
#                         gnb= 0: Return background as b and W
#                         gnb=-1: Return full rank background B
#                         gnb<-1: Don't return background
nb_patch = 0        # number of background components (rank) per patch if gnb>0,
#                     else it is set automatically
min_corr = .8       # min peak value from correlation image
min_pnr = 10        # min peak to noise ration from PNR image
ssub_B = 2          # additional downsampling factor in space for background
ring_size_factor = 1.4  # radius of ring is gSiz*ring_size_factor

opts.change_params(params_dict={'method_init': 'corr_pnr',  # use this for 1 photon
                                'K': K,
                                'gSig': gSig,
                                'gSiz': gSiz,
                                'merge_thresh': merge_thresh,
                                'p': p,
                                'tsub': tsub,
                                'ssub': ssub,
                                'rf': rf,
                                'stride': stride_cnmf,
                                'only_init': True,    # set it to True to run CNMF-E
                                'nb': gnb,
                                'nb_patch': nb_patch,
                                'method_deconvolution': 'oasis',       # could use 'cvxpy' alternatively
                                'low_rank_background': low_rank_background,
                                'update_background_components': True,  # sometimes setting to False improve the results
                                'min_corr': min_corr,
                                'min_pnr': min_pnr,
                                'normalize_init': False,               # just leave as is
                                'center_psf': True,                    # leave as is for 1 photon
                                'ssub_B': ssub_B,
                                'ring_size_factor': ring_size_factor,
                                'del_duplicates': True,                # whether to remove duplicates from initialization
                                'border_pix': bord_px})                # number of pixels to not consider in the borders)

# compute some summary images (correlation and peak to noise)
#cn_filter, pnr = cm.summary_images.correlation_pnr(images, gSig=gSig[0], swap_dim=False) # change swap dim if output looks weird, it is a problem with tiffile

      136133 [params.py:                 set():780] [46077] Changing key rf in group patch from None to 40
      136134 [params.py:                 set():780] [46077] Changing key stride in group patch from None to 20
      136135 [params.py:                 set():780] [46077] Changing key nb_patch in group patch from 1 to 0
      136136 [params.py:                 set():780] [46077] Changing key low_rank_background in group patch from True to None
      136140 [params.py:                 set():780] [46077] Changing key del_duplicates in group patch from False to True
      136141 [params.py:                 set():780] [46077] Changing key p in group preprocess from 2 to 1
      136142 [params.py:                 set():780] [46077] Changing key method_init in group init from greedy_roi to corr_pnr
      136143 [params.py:                 set():780] [46077] Changing key K in group init from 30 to None
      136145 [params.py:                 set():780] [46077] Changing key gSig in group

In [8]:
#inspect_correlation_pnr(cn_filter, pnr)

In [9]:
# print parameters set above, modify them if necessary based on summary images
print(min_corr) # min correlation of peak (from correlation image)
print(min_pnr)  # min peak to noise ratio

0.8
10


In [10]:
cnm = cnmf.CNMF(n_processes=n_processes, dview=dview, Ain=Ain, params=opts)

In [11]:
print('running cnmfe')
cnm.fit(images)

/Users/johnmarshall/miniconda2/envs/caiman/lib/python3.6/site-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
      210880 [params.py:                 set():780] [46077] Changing key init_batch in group online from 200 to 202
      210881 [cnmf.py:                 fit():404] [46077] (202, 480, 752)
      210882 [params.py:                 set():780] [46077] Changing key medw in group spatial from None to (3, 3)
      210918 [params.py:                 set():780] [46077] Changing key se in group spatial from None to [[1 1 1]
 [1 1 1]
 [1 1 1]]
      210919 [params.py:                 set():780] [46077] Changing key ss in group spatial from None to [[1 1 1]
 [1 1 1]
 [1 1 1]]
      21092

running cnmfe
(80, 80)
1681.869115114212
Transforming patches into full matrix
Skipped %d Empty Patch 0
Generating background
Generating background DONE
merging


/Users/johnmarshall/miniconda2/envs/caiman/lib/python3.6/site-packages/caiman/source_extraction/cnmf/merging.py:141: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  C_corr[ii, overlap_indeces] = corr_values


[8 9]
[3 7]


/Users/johnmarshall/miniconda2/envs/caiman/lib/python3.6/site-packages/scipy/signal/spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 202, using nperseg = 202
  .format(nperseg, input_length))
/Users/johnmarshall/miniconda2/envs/caiman/lib/python3.6/site-packages/caiman/source_extraction/cnmf/deconvolution.py:991: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  g = np.linalg.lstsq(A, xc[lags + 1:])[0]


No neurons merged!


In [12]:
# get contours of found components
Cn = cm.local_correlations(images.transpose(1,2,0))
Cn[np.isnan(Cn)] = 0

In [13]:
#%% COMPONENT EVALUATION
# the components are evaluated in three ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient
#   c) each shape passes a CNN based classifier

min_SNR = 3            # adaptive way to set threshold on the transient size
r_values_min = 0.85    # threshold on space consistency (if you lower more components
#                        will be accepted, potentially with worst quality)
cnm.params.set('quality', {'min_SNR': min_SNR,
                           'rval_thr': r_values_min,
                           'use_cnn': False})
cnm.estimates.evaluate_components(images, cnm.params, dview=dview)

print(' ***** ')
print('Number of total components: ', len(cnm.estimates.C))
print('Number of accepted components: ', len(cnm.estimates.idx_components))

     2192287 [params.py:                 set():780] [46077] Changing key min_SNR in group quality from 2.5 to 3
     2192289 [params.py:                 set():780] [46077] Changing key rval_thr in group quality from 0.8 to 0.85
     2192291 [params.py:                 set():780] [46077] Changing key use_cnn in group quality from True to False
     2192296 [components_evaluation.py:estimate_components_quality():682] [46077] EVALUATING IN PARALLEL... NOT RETURNING ERFCs


 ***** 
Number of total components:  13
Number of accepted components:  13


In [14]:
hf = h5py.File(folder + 'output_data.h5', 'w')
hf.create_dataset('A', data=cnm.estimates.A.toarray())
hf.create_dataset('C', data=cnm.estimates.C)
hf.close()

In [15]:
#plt.plot(cnm.estimates.C[0])
#plt.show()

     2282918 [_base.py:_update_title_position():2515] [46077] update_title_pos
     2282936 [font_manager.py:    _findfont_cached():1254] [46077] findfont: Matching :family=sans-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0 to DejaVu Sans ('/Users/johnmarshall/miniconda2/envs/caiman/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf') with score of 0.050000.
